# Assignment 2

To be delivered until 2022/01/24 23:59:59.

## 1) Arduino

You will start by setting up a series of connections in order to extract some data with the Arduino. First make the connections as shown below. **Mind the direction of the temperature sensor. If you have an incorrect position, you will be connection the power to the ground and vice-versa and you will damage the sensor.** The photoresistor sensor on the other hand has no polarity.

<img src="temp_photo_cropped.png" width=1000/>

On this problem, you will read temperature and luminance from the sensors and print them on the serial.

**1)** Code an Arduino sketch, where the value of temperature and luminance are printed to the serial. For each serial print that you make, print the value of temperature, then a semicolon, then the value of luminance with a new line (use no whitespaces). You can do this by using three separate `Serial.print`, with the last one being a `Serial.println`. Print values 5 times per second (use the delay function to control this). Manually influence the readings of the sensors, by covering the photoresistor or shining light on it, and by lightly and carefully touching the temperature sensor to increase its temperature readings.

**Note that the temperature sensor appears not to be very reliable. Since the objective of this exercise is just to plot the results, this should not be an issue.**

**Copy and paste your arduino code below. You may use a python code cell, even though the code can not be run.**

*Hint: for the temperature value to be in celsius, divide the read value by 1024 and multiply it by 500. The luminance does not have to be converted*

To import the data into Arduino, keep it running (the Serial Monitor must be closed in Arduino) and run the following code. Change the COM port to your own. This block of code will read 1000 values from the Serial. Given that each observation is taken every 0.2 seconds, it should take a minute and a half.

In [28]:
import serial
import time

ser = serial.Serial('COM4', 9600, timeout=1)
time.sleep(2)

data = []
for i in range(500):
    line = ser.readline()
    if line:
        string = line.decode()
        data.append(string)

ser.close()

Convert the data into a pandas dataframe and save it in a csv file. Besides the value of temperature and luminance, also include the time, considering the first observation at $t=0$ and every observation 0.2 seconds after the previous one. **The file must be submitted in Fenix and included in your Github repo**.

Plot the Temperature against time, the luminance against time and the temperature against the luminance.

## 2) Databases

For the databases part of this assignment, you will use the mimic-iii database from the laboratory session. Start by adding a few new tables to the database, using the SQL files included in the assignment's files. Open PGAdmin and connect to your mimic-iii database. **To properly load these tables, load the following files exactly and by the order presented.**

1) Run demographic.sql

2) Run lab_firstday.sql

You will now have to answer a few SQL questions.

**1.** Open the connection to your mimic-iii database. If you want, you can delete your credentials before submitting the assignment, but if you do so, please run the notebook first, for the results to be displayed.

In [141]:
import pandas as pd
import numpy as np
import psycopg2 as psql
import matplotlib.pyplot as plt

conn = psql.connect(host='localhost',
                    database='mimic-iii',
                    user='username',
                    password='password',
                    port=5432)

cursor = conn.cursor()
cursor.execute("select version()")
data = cursor.fetchone()

print("Connection established to: ", data)

Connection established to:  ('PostgreSQL 14.1, compiled by Visual C++ build 1914, 64-bit',)


**2.** Create a function that receives an SQL query and automatically opens a cursor, queries the database, extracts the columns, creates a pandas database, and closes the connections.

In [142]:
import pandas as pd
import psycopg2 as psql
def receive_query( query ):
    cursor = conn.cursor()
    cursor.execute(query)
    colnames = [desc[0] for desc in cursor.description]
    data = cursor.fetchall()
    query_table = pd.DataFrame(data, columns = colnames)
    print(query_table)
    conn.close()
    return

**3.** Query the table admissions filtering for admission type as elective and insurance as private.

In [143]:
filtering = "SELECT * FROM public.admissions WHERE admission_type = 'ELECTIVE' AND insurance = 'Private'"
cursor.execute(filtering)

colnames = [desc[0] for desc in cursor.description]
data = cursor.fetchall()



filtered_table = pd.DataFrame(data, columns = colnames)
filtered_table

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12315,10065,183314,2189-09-08 07:15:00,2189-09-20 14:00:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,ENGL,CATHOLIC,SINGLE,WHITE,None,None,ESOPHAGEAL CANCER/SDA,0,1
1,40972,43798,130870,2198-06-29 07:15:00,2198-08-07 23:59:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,CATHOLIC,MARRIED,WHITE,None,None,ESOPHAGEAL CA/SDA,0,1
2,41053,44083,125157,2112-05-04 08:00:00,2112-05-11 14:15:00,None,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,ENGL,CATHOLIC,SINGLE,WHITE,None,None,ESOPHAGEAL CA/SDA,0,1


**4.** Query the table admissions, filtering for the same conditions as the previous exercise (admission type as elective and insurance as private). Join the "drgcodes" table on the admission ID. Display only the columns regarding the subject id, admission id, time of death, and description of the drug.

In [144]:
joined = "SELECT admissions.subject_id, admissions.hadm_id as admission_type, deathtime, drgcodes.description FROM admissions LEFT JOIN drgcodes ON admissions.hadm_id = drgcodes.hadm_id WHERE insurance = 'Private' and admission_type = 'ELECTIVE';"
cursor.execute(joined)

colnames = [desc[0] for desc in cursor.description]
data2 = cursor.fetchall()


joined_table = pd.DataFrame(data2, columns = colnames)
joined_table

,subject_id,admission_type,deathtime,description
0,10065,183314,None,"Major Stomach, Esophageal & Duodenal Procedures"
1,10065,183314,None,"STOMACH, ESOPHAGEAL & DUODENAL PROC AGE >17 W ..."
2,43798,130870,None,Tracheostomy W Long Term Mechanical Ventilatio...
3,43798,130870,None,Tracheostomy W Long Term Mechanical Ventilatio...
4,43798,130870,None,"TRACH W MV 96+ HRS OR PDX EXC FACE, MOUTH & NE..."
5,44083,125157,None,"Major Stomach, Esophageal & Duodenal Procedures"
6,44083,125157,None,"Major Stomach, Esophageal & Duodenal Procedures"
7,44083,125157,None,"STOMACH, ESOPHAGEAL & DUODENAL PROC W MCC"


**5.1.** Obtain the dataset for this problem, by running the SQL query below.

In [145]:
dataset = "SELECT pivoted_lab.*," +\
                "gender as gender," +\
                "admission_age," +\
                "ethnicity_grouped as eth_grp," +\
                "hospital_expire_flag," +\
                "los_icu " +\
        "FROM demographics " +\
        "LEFT JOIN pivoted_lab " +\
        "ON demographics.icustay_id = pivoted_lab.icustay_id " +\
        "WHERE first_icu_stay = true"
cursor.execute(dataset)

colnames = [desc[0] for desc in cursor.description]
data3 = cursor.fetchall()


dataset_table = pd.DataFrame(data3, columns = colnames)
dataset_table

,subject_id,hadm_id,icustay_id,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,...,sodium_max,bun_min,bun_max,wbc_min,wbc_max,gender,admission_age,eth_grp,hospital_expire_flag,los_icu
0,10006,142345,206504,12.0,20.0,2.7,3.4,NaN,NaN,29.0,...,139.0,9.0,11.0,4.6,7.8,F,70.0,black,0,1.0
1,10011,105331,232110,12.0,12.0,2.6,2.6,2.0,2.0,23.0,...,136.0,3.0,3.0,10.6,10.6,F,36.0,unknown,1,13.0
2,10013,165520,264446,13.0,13.0,NaN,NaN,13.0,13.0,29.0,...,138.0,32.0,32.0,13.8,16.2,F,87.0,unknown,1,2.0
3,10017,199207,204881,13.0,13.0,2.8,2.8,NaN,NaN,29.0,...,139.0,3.0,3.0,15.8,15.8,F,74.0,white,0,2.0
4,10019,177759,228977,20.0,46.0,3.2,3.2,NaN,NaN,10.0,...,141.0,31.0,53.0,3.7,6.8,M,49.0,white,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,44083,198330,286428,16.0,16.0,NaN,NaN,NaN,NaN,21.0,...,142.0,12.0,12.0,12.3,14.9,M,55.0,white,0,3.0
124,44154,174245,217724,15.0,15.0,NaN,NaN,NaN,NaN,19.0,...,142.0,16.0,21.0,12.2,17.1,M,300.0,white,1,0.0
125,44212,163189,239396,15.0,21.0,2.9,3.0,NaN,NaN,18.0,...,150.0,37.0,57.0,8.8,11.4,F,45.0,black,0,31.0
126,44222,192189,238186,11.0,15.0,NaN,NaN,NaN,NaN,22.0,...,135.0,21.0,24.0,9.3,9.9,M,73.0,white,0,1.0


**5.2.** Close the connection to your SQL server.

In [147]:
conn.close()

**5.3.** Prepare your dataset:

* Drop the ID columns of subject, admission and ICU stay.
* Drop columns with at least one NA value.
* Encode the categorical columns, the ethnicity and gender ('eth_grp', 'gender'). *Suggestion: use pd.get_dummies*

In [146]:
dummies = pd.get_dummies(dataset_table[['eth_grp', 'gender']])
dataset_table = pd.concat([dataset_table, dummies[['eth_grp_asian', 'eth_grp_black' ,'eth_grp_hispanic', 'eth_grp_native', 'eth_grp_other', 'eth_grp_unknown', 'eth_grp_white', 'gender_F', 'gender_M']]], axis=1)
dataset_table.drop(['subject_id', 'hadm_id', 'icustay_id','eth_grp', 'gender'], inplace=True, axis=1)
dataset_table = dataset_table.dropna(axis='columns')
dataset_table

,bicarbonate_min,bicarbonate_max,creatinine_min,creatinine_max,chloride_min,chloride_max,glucose_min,glucose_max,hematocrit_min,hematocrit_max,...,los_icu,eth_grp_asian,eth_grp_black,eth_grp_hispanic,eth_grp_native,eth_grp_other,eth_grp_unknown,eth_grp_white,gender_F,gender_M
0,29.0,31.0,3.0,3.5,96.0,100.0,84.0,217.0,36.9,42.4,...,1.0,0,1,0,0,0,0,0,1,0
1,23.0,23.0,0.7,0.7,107.0,107.0,79.0,79.0,33.9,34.0,...,13.0,0,0,0,0,0,1,0,1,0
2,29.0,29.0,1.7,1.7,98.0,100.0,134.0,165.0,28.1,29.2,...,2.0,0,0,0,0,0,1,0,1,0
3,29.0,29.0,0.3,0.3,100.0,100.0,137.0,137.0,27.5,27.5,...,2.0,0,0,0,0,0,0,1,1,0
4,10.0,18.0,4.0,7.2,83.0,104.0,80.0,360.0,30.6,36.0,...,1.0,0,0,0,0,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,21.0,21.0,0.7,0.7,108.0,108.0,151.0,151.0,26.0,29.0,...,3.0,0,0,0,0,0,0,1,0,1
124,19.0,19.0,0.9,0.9,107.0,113.0,164.0,177.0,40.8,41.8,...,0.0,0,0,0,0,0,0,1,0,1
125,18.0,23.0,3.0,4.8,108.0,115.0,99.0,122.0,23.8,25.9,...,31.0,0,1,0,0,0,0,0,1,0
126,22.0,27.0,1.2,1.7,100.0,101.0,56.0,268.0,37.8,39.0,...,1.0,0,0,0,0,0,0,1,0,1


**6.** Fit the following tree-based classifiers to the dataset. For each method:

* Perform k-fold cross validation to evaluate the models. Consider 10 folds.

* Plot the ROC curves for each fold, along with the mean ROC curve.

* Calculate the mean AUC.

**a.** Decision tree.

**b.** Random forest

**c.** Gradient Boosting

**7.1.** Perform a grid search cross-validation on the Gradient boosting methods, changing the value of the learning rate (0.01 to 0.5) and the number of estimators (50-500). Consider the mean AUC of the folds as the performance measure.

**7.2.** Plot a scatterplot of the learning rate versus the number of estimators, with the mean AUC as the color gradient.

**8.1.** Perform forward stepwise selection on the dataset. Use the best parameters of the gradient boosting method obtained in **7.1.**.

**8.2.** Compare and comment the results from **8.1.** with the features importance obtained through the grid search of queastion **7.1.**.

## 3) Theoretical Questions

**1.** Consider a dataset where best subset, forward stepwise and backward stepwise selection will be performed. For each of the 3 approaches, we obtain $p+1$ models, $p$ being the total number of predictors. This means that each approach has a model with 0 predictors, one with 1 predictor, one with 2 predictor, up until one model with $p$ predictors. Answer and justify the following questions:

**a)** Which of the three models with $k, \, \forall_{k \in [0,p]}$ predictors has the smallest training RSS?

**b)** Which of the three models with $k, \, \forall_{k \in [0,p]}$ predictors has the smallest test RSS?

**c)** Evaluate the following statements with *true* or *false*. Justify your answers.

    i. The predictors in the k-variable model identified by forward stepwise selection are a subset of the predictors in the (k+1)-variable model identified by forward stepwise selection.

    ii. The predictors in the k-variable model identified by backward stepwise selection are a subset of the predictors in the (k + 1)-variable model identified by backward stepwise selection.

    iii. The predictors in the k-variable model identified by backward stepwise selection are a subset of the predictors in the (k + 1)-variable model identified by forward stepwise selection.

    iv. The predictors in the k-variable model identified by forward stepwise selection are a subset of the predictors in the (k+1)-variable model identified by backward stepwise selection.

    v. The predictors in the k-variable model identified by best subset selection are a subset of the predictors in the (k + 1)-variable model identified by best subset selection.

**2.** Ridge regression tends to give similar coefficient values to correlated variables, whereas lasso regression may give substantially different coefficients to correlated variables. This questions explores this property in a simplified setting.

Suppose that $n=2$, $p=2$, $x_{11} = x_{12}$, $x_{21} = x_{22}$. Moreover, suppose that $y_1 + y_2 = 0$ and $x_{11} + x_{21} = 0$ and $x_{12} + x_{22} = 0$, meaning that the estimate for the intercept in a least squares, ridge regression, or lasso regression is zero: $\hat{\beta} = 0$.

**a)** Write the ridge regression optimization problem in this setting.

**b)** Prove that in this setting, the ridge regression coefficient estimates satisfy $\hat{\beta}_1 = \hat{\beta}_2$.

**c)** Write the lasso regression optimization problem in this setting.

**d)** Prove that in this setting, the lasso regression coefficients $\hat{\beta}_1$ and $\hat{\beta}_2$ are not unique, meaning that there are many possible solutions to the optimization problem in (c). Describe these solutions.

**3.** Draw an example of a partition of two-dimensional feature space that could result from recursive binary splitting. Your example should contain at least six regions. Draw a decision tree corresponding to this partition. Be sure to label all aspects of your figures, including the regions R1, R2,..., the cutpoints t1, t2,..., and so forth.

If you prefer you can draw it by hand or in any software and use a scan of it.

**4.** In 2 dimensions, a linear decision boundary takes the form $\beta_0 + \beta_1 X_1 + \beta_2 X_2 = 0$. Consider a nn-linear decision boundary:

**a)** Sketch the curve

$$(1 + X_1)^2 + (2 - X_2)^2 = 4$$

Additionally, indicate on your sketch the set of points that verify the condition

$$(1 + X_1)^2 + (2 - X_2)^2 > 4$$

and the condition

$$(1 + X_1)^2 + (2 - X_2)^2 \leq 4$$

**b)** Suppose that a classifier assigns an observation to the blue class if $(1 + X_1)^2 + (2 - X_2)^2 > 4$ and to the red class otherwise. To what class are the following observations classified? (0,0), (-1,1), (2,2), (3,8)

**c)** Prove that while the decision boundary in (b) is not linear in terms of $X_1$ and $X_2$, it is linear in terms of $X_1$, $X_1^2$, $X_2$, and $X_2^2$.

# 4) Laboratory Questions

What are the advantages and disadvantages of relational dabases versus graph databases, and when should one type be preferred over the other?